# Exercise 2 -- reclustering

Now you should know the drill. The first part is exactly the same

In [2]:
import FWCore.ParameterSet.Config as cms
from Configuration.StandardSequences.Eras import eras
process = cms.Process("USER",eras.Run2_2018)

## Load services
process.load("Configuration.Geometry.GeometryRecoDB_cff")
process.load("Configuration.StandardSequences.FrontierConditions_GlobalTag_cff")
process.load("Configuration.StandardSequences.MagneticField_cff")
process.load("FWCore.MessageService.MessageLogger_cfi")
process.MessageLogger.cerr.FwkReport.reportEvery = 10
from Configuration.AlCa.GlobalTag import GlobalTag
process.GlobalTag = GlobalTag(process.GlobalTag, '102X_upgrade2018_realistic_v15')

##Inputs
#Source
process.source = cms.Source(
    "PoolSource",
    fileNames = cms.untracked.vstring(
        'root://cmseos.fnal.gov//store/user/hats/2019/Tagging/TTToSemiLeptonic_RunIIAutumn18MiniAOD-102X_upgrade2018_realistic_v15/D7F68ED3-7D8A-0A4E-A53E-D5557F11760C.root'
    )
)

#Events to run
process.maxEvents = cms.untracked.PSet( 
    input = cms.untracked.int32(100) 
)

#Long summary
process.options = cms.untracked.PSet( 
    wantSummary = cms.untracked.bool(True) 
)


## Re-making jets

In [3]:
#################################################
## Remake jets
#################################################

## Filter out neutrinos from packed GenParticles
process.packedGenParticlesForJetsNoNu = cms.EDFilter(
    "CandPtrSelector", 
    src = cms.InputTag("packedGenParticles"), 
    cut = cms.string("abs(pdgId) != 12 && abs(pdgId) != 14 && abs(pdgId) != 16")
)

## Define GenJets
from RecoJets.JetProducers.ak4GenJets_cfi import ak4GenJets
process.ak4GenJetsNoNu = ak4GenJets.clone(
    src = 'packedGenParticlesForJetsNoNu'
)

## Select charged hadron subtracted packed PF candidates
process.pfCHS = cms.EDFilter(
    "CandPtrSelector", 
    src = cms.InputTag("packedPFCandidates"), 
    cut = cms.string("fromPV")
)

from RecoJets.JetProducers.ak4PFJets_cfi import ak4PFJets
## Define PFJetsCHS
process.ak4PFJetsCHS = ak4PFJets.clone(
    src = 'pfCHS', 
    doAreaFastjet = True
)

## Making PAT jets

The following part produces the `pat::Jet` collection out of the newly created `reco::Jet` collection. As for the previous exercise, we will use a PAT tool modifier: `switchJetCollection`. You can view all its options as usual.

In [4]:
from PhysicsTools.PatAlgos.tools.jetTools import switchJetCollection
print switchJetCollection.__doc__
for par_name, par in switchJetCollection._parameters.iteritems():
    print '   - %s:  %s' % (par_name, par.description)


    Tool to switch parameters of the PAT jet collection to your PAT Tuple.
    
   - postfix:  postfix from usePF2PAT
   - btagPrefix:  Prefix to be added to b-tag discriminator and TagInfo names
   - jetSource:  Label of the input collection from which the new patJet collection should be created
   - pfCandidates:  Label of the input collection for candidatecandidatese used in b-tagging
   - explicitJTA:  Use explicit jet-track association
   - pvSource:  Label of the input collection for primary vertices used in b-tagging
   - svSource:  Label of the input collection for IVF vertices used in b-tagging
   - elSource:  Label of the input collection for electrons used in b-tagging
   - muSource:  Label of the input collection for muons used in b-tagging
   - runIVF:  Re-run IVF secondary vertex reconstruction
   - tightBTagNTkHits:  Enable legacy tight b-tag track selection
   - loadStdRecoBTag:  Load the standard reconstruction b-tagging modules
   - svClustering:  Secondary vertices 

We want to make a new pat jet collection from the jet collection we just created, and add the following discriminators: CSVv2, cMVAv2, and DeepCSV. Please fill the following code

In [6]:
## b-tag discriminators
bTagDiscriminators = [
    #
    # PUT THE DISCRIMINATORS HERE
    #
]

## Switch the default PAT jet collection to the above-defined ak4PFJetsCHS
#
# REPLACE THE FIXME!
# 
FIXME = 'FIXME' #this is just here to avoid python to raise an exception
switchJetCollection(
    process,
    jetSource = FIXME,
    pvSource = FIXME,
    pfCandidates = FIXME,
    svSource = FIXME,
    muSource = FIXME,
    elSource = FIXME,
    btagDiscriminators = bTagDiscriminators,
    jetCorrections = ('AK4PFchs', ['L1FastJet', 'L2Relative', 'L3Absolute'], 'None'),
    genJetCollection = FIXME,
    genParticles = FIXME
)
getattr(process,'selectedPatJets').cut = cms.string('pt > 10')   # to match the selection for slimmedJets in MiniAOD

TypeError: The type for parameter "FIXME" is not "<class 'FWCore.ParameterSet.Types.InputTag'>"

What is this needed for?

In [5]:
from PhysicsTools.PatAlgos.tools.pfTools import adaptPVs
## Adapt primary vertex collection
adaptPVs(
    process, 
    pvCollection=cms.InputTag('offlineSlimmedPrimaryVertices')
)

Switching PV collection for PF2PAT: cms.InputTag("offlineSlimmedPrimaryVertices")
***********************************


Now, set the output. Please note the use of the `Task`, which allows the configuration file to run smoothly in the unscheduled mode

In [6]:
#output
process.out = cms.OutputModule(
    "PoolOutputModule",
    fileName = cms.untracked.string('recluster_jets.root'),
    ## save only events passing the full path
    #SelectEvents = cms.untracked.PSet( SelectEvents = cms.vstring('p') ),
    outputCommands = cms.untracked.vstring(
        'drop *', ## Do not keep anything
        'keep *_slimmedJets_*_*', 
        'keep *_selectedPatJets_*_*', 
    )
)

#
# ADD YOUR CODE HERE!
#
process.outpath = cms.EndPath(process.out, patAlgosToolsTask)